In [1]:
import json
import pickle
from itertools import islice

from champs import top_champs

# Open the pkl file in read binary mode
with open('full_result.pickle', 'rb') as f:
    data = pickle.load(f)

In [17]:
my_champ = "Garen"

In [3]:
data[my_champ]

{'Camille': '0.0512',
 'Olaf': '0.0454',
 'Yone': '0.0427',
 'Gnar': '0.0385',
 'Irelia': '0.0374',
 'Shen': '0.0341',
 'Shyvana': '0.0267',
 'Jayce': '0.0242',
 'Mordekaiser': '0.0226',
 'Darius': '0.0179',
 "K'Sante": '0.0163',
 'Yorick': '0.0156',
 'Gangplank': '0.015',
 'Teemo': '0.01',
 'Malphite': '-0.1071',
 'Fiora': '-0.0437',
 'Kayle': '-0.0307',
 'Volibear': '-0.0295',
 'Sion': '-0.0261',
 'Rumble': '-0.0251',
 'Nasus': '-0.023',
 'Pantheon': '-0.0151',
 'Gwen': '-0.0148',
 'Garen': '-0.0129',
 'Renekton': '-0.0129'}

In [4]:
results = {}
for c in data:
    garen_data = data[my_champ].copy()
    for k, v in garen_data.items():
        if float(v) < 0:
            garen_data[k] = 0 if float(data[c].get(k, 0)) > 0 else float(garen_data[k])
            # if float(data[c].get(k, 0)) > 0:
            #     print(f"{c} counters weakness {k}")

    results[c] = 0
    for value in garen_data.values():
        if float(value) < 0:
            results[c] += 1

In [5]:
results = dict(sorted(results.items(), key=lambda item: item[1]))

for c in results:
    print(f"{c}: {results[c]}")

Ornn: 4
Dr. Mundo: 5
Darius: 6
Gnar: 6
Illaoi: 6
Shen: 6
Singed: 6
Aatrox: 7
Cho'Gath: 7
Fiora: 7
Gangplank: 7
Malphite: 7
Mordekaiser: 7
Nasus: 7
Pantheon: 7
Riven: 7
Sett: 7
Teemo: 7
Warwick: 7
Camille: 8
Gragas: 8
Irelia: 8
Jax: 8
Poppy: 8
Sion: 8
Tryndamere: 8
Kennen: 9
K'Sante: 9
Quinn: 9
Renekton: 9
Trundle: 9
Urgot: 9
Yorick: 9
Akali: 10
Garen: 10
Jayce: 10
Volibear: 10
Wukong: 10
Kled: 11
Maokai: 11
Rengar: 11
Vladimir: 11
Yasuo: 11
Zac: 11


In [18]:
def weak_matrix(champ):
    results = {}
    for c in data:
        garen_data = data[champ].copy()
        for k, v in garen_data.items():
            if float(v) < 0:
                garen_data[k] = 0 if float(data[c].get(k, 0)) > 0 else float(garen_data[k])
                # if float(data[c].get(k, 0)) > 0:
                #     print(f"{c} counters weakness {k}")
    
        results[c] = 0
        for value in garen_data.values():
            if float(value) < 0:
                results[c] += 1
        results = dict(sorted(results.items(), key=lambda item: item[1]))
    return dict(islice(results.items(), 7))

In [21]:
def score(champ1, champ2):
    results = {}
    garen_data = data[champ1].copy()
    for k, v in garen_data.items():
        if float(v) < 0:
            garen_data[k] = 0 if float(data[champ2].get(k, 0)) > 0 else float(garen_data[k])
            # if float(data[c].get(k, 0)) > 0:
            #     print(f"{c} counters weakness {k}")

    results[champ2] = 0
    for value in garen_data.values():
        if float(value) < 0:
            results[champ2] += 1
    results = dict(sorted(results.items(), key=lambda item: item[1]))
    return dict(islice(results.items(), 7))

In [20]:
wm = weak_matrix(my_champ)
wm

{'Malphite': 8,
 'Kled': 9,
 'Ornn': 9,
 'Sion': 9,
 'Dr. Mundo': 10,
 'Jax': 10,
 'Nasus': 10}

In [30]:
def get_best_third_champ(champ):
    results = {}
    for c in weak_matrix(champ):
        batch = {}
        for c2 in weak_matrix(c):
            m = score(champ, c2)
            for k, v in m.items():
                batch[k] = v
        results[c] = batch
    return results

In [52]:
def evaluate_triplets(results, champ):
    final = {}
    for c in results:
        sums = {}
        for c2 in results[c]:
            sums[c2] = results[c][c2] + wm.get(c, 100)
            
        min_value = min(sums.values())
        min_keys = [key for key, value in sums.items() if value == min_value]
    
        final[c] = {}
        for k in min_keys:
            final[c][k] = sums[k]

    print(f"These are your best 3-trick combos for {champ}:\n")
    for c in final:
        for c2 in final[c]:
            print(f"- {champ}, {c}, {c2} (Score: {final[c][c2]})")
    print("\n* Score stands for amount of counters a combo has. Lower is better.")

In [53]:
evaluate_triplets(get_best_third_champ(my_champ), my_champ)

These are your best 3-trick combos for Garen:

- Garen, Malphite, Sett (Score: 19)
- Garen, Malphite, Gangplank (Score: 19)
- Garen, Kled, Ornn (Score: 18)
- Garen, Ornn, Kled (Score: 18)
- Garen, Sion, Jax (Score: 19)
- Garen, Sion, Nasus (Score: 19)
- Garen, Dr. Mundo, Kled (Score: 19)
- Garen, Jax, Sion (Score: 19)
- Garen, Jax, Ornn (Score: 19)
- Garen, Nasus, Ornn (Score: 19)

* Score stands for amount of counters a combo has. Lower is better.


In [1]:
# for c in top_champs:
#     evaluate_triplets(get_best_third_champ(c), c)